# Loading a Design Scenario

```{warning}
The following is a work in progress. More content will be added as produced.
```

*Rangekeeper* provides API access to [Speckle](https://speckle.systems/). This
enables loading and extending a 3D design scenario with *Rangekeeper* to produce
its financial valuation, as well as execute any automated decision-making or
optimisation processes, and sending the results back to Speckle.

## Object Model
Before walking through the I/O methods, it is important to understand the
requirements *Rangekeeper* places on how a design scenario must be structured,
so that the `Flow`s and `Stream`s of a DCF Proforma can be appropriately
attributed to the design scenario's objects (especially its representations of
physical spaces, components, and equipment).

### Elements and Relationships
A holistic representation of a real estate development scenario is that it is a
web of inter-related 'elements'; where each element is a 'thing' and can have
multiple relationships with other elements. For example, a floor is a type
of element (say, a 'Space') and it has relationships with floors above and
below it, as well as relationships to its sub-elements (e.g. rooms), or even
what services it (e.g. the mechanical systems controlling its air conditioning)

```{figure} resources/devModelGraph.jpg
---
width: 100%
name: Real Estate Scenario as a Graph
---
An entire Real Estate Development Scenario is represented as a huge web (graph)
of `Element`s and `Relationship`s
```

#### `Element`s
An `Element` is a ‘thing’ that has substance, or is composed of sub-`Element`s
that have substance, material, mass, volume, existence, are ‘real’, etc.

#### `Relationship`s
`Relationship`s describe some sort of (directed) association, connection, or
link between `Element`s. Each `Relationship` is described by its ‘type’; for
instance, an `Element` can be related to another by virtue of it “being
contained (spatially)” by the other, or it could be “installed before” another
`Element`.
There can be any number of `Relationship`s between any number of `Element`s.

#### `Assembly`s
In order to clearly encapsulate ‘groups’ of intuitively-related `Element`s,
*Rangekeeper* has a concept of an `Assembly`, which is an object that defines
a collection of `Relationship`s and their associated `Element`s.
`Assembly`s are `Element`s themselves, too –- so they can also be related to
other `Element`s.
This means that *Rangekeeper* can traverse from `Assembly` to `Assembly` through
overlapping `Element`s, in a similar fashion to common-sense conceptualisation
of how a real estate development scenario is structured.

```{figure} resources/devModelAssembly.jpg
---
width: 100%
name: An Assembly of Elements
---
An `Assembly` is a non-exclusive collection of `Element`s and `Relationship`s,
enabling the traversal of `Element`s in the graph.
```

### Definitions
#### Properties of `Element`s
All `Element`s have the following properties:
1. **Id**: a unique, immutable, identifier
2. **Name**: a string
3. **Type**: an `ElementType`, which is a node of a tree of `ElementType`s)
4. **Attributes**: a key-value store of specific properties of the `Element`
   (eg, its area, geometry, material, etc).
There are two key `Attribute`s (stored in the `Attributes` property), where
*Rangekeeper* seeks and records data for its use:
   -  **Events**: a set of time-based occurrences, or experiences, that happen
      in or to the `Element` (called `Event`s)
   -  **Measurements**: a key-value store of named quantities measuring the
       `Element` (e.g. its Net Lettable Area (NLA))

In addition, `Assembly`s have properties collecting related `Element`s:
1. **ElementIds**: a set pointers to `Element`s
2. **Relationships**: a set of `Relationship`s

#### Properties of `Relationship`s
Meanwhile, all `Relationship`s have the following properties:
1. **Type**: a `RelationshipType`, which is an item from a list of defined
   `RelationshipType`s
2. **SourceId**: the `Element` `Id` that is the source of the `Relationship`
3. **TargetId**: the `Element` `Id` that is the target of the `Relationship`

#### `Event`s
Every `Element` can exist in both space and time. Most `Element`s will have (or
its sub-`Element`s will have, in the case of an `Assembly`) geometry/s that
define the extent of space that the Element exists in.

To represent how an `Element` exists in time, the `Element` has `Event`s
recorded against it. For example, an `Event` may record the installation of the
`Element` during construction, or it may record production of revenue during
operation of the real estate project as an asset.

`Flow`s and `Stream`s are subclasses of `Event`s.

```{figure} resources/objModel.jpg
---
width: 100%
name: UML-style diagram of foundation of Rangekeeper's object model
---
Diagram of general overview of the founational concepts of *Rangekeeper*'s
object model, in UML-style.
```

## Rationale and Implications
*Rangekeeper* will use the `Relationship`s defined in the `Assembly`/s of a
design scenario to structure the "drill-downs" and "roll-ups" (ie, the
compositions) of `Flow`s into `Stream`s that are appropriate for the kinds of
summations and metrics that the DCF Proforma model requires.

For example, an office building may produce revenue from multiple tenants. A
tenant may occupy multiple floors, or a part (space) of a single floor.
For *Rangekeeper*, each space would be an `Element`; with its `Events` property
containing a set of `Flow`s and/or `Stream`s that represent revenues or costs
associated with that space. Those may be aggregated into cash flows generated by
the tenant of those spaces, and analysed as such, or they may be aggregated per
floor, or per building, or per project.

This makes it simple to calculate things like the share of total revenue
generated by each tenant; or the share of revenue generated by each floor.
Likewise with costs.

The multi-faceted nature of the `Element`-`Relationship` model enables multiple
pathways for those aggregations to be specified, and thus enables an efficient
and broad ability to query the model for the kinds of analyses pertinent to
real estate valuation.

### Technicalities
To provide a *Rangekeeper* model with the `Element`-`Relationship` organization,
those need to either be defined in the design scenario itself, or they need to
be specified after receiving and loading the design scenario.

#### Defining `Element`s and `Relationship`s in the Design Scenario
*Rangekeeper* uses the [Speckle](https://speckle.systems) service and data model
for the exchange of design scenarios, and scaffolds `Element`s off a [Speckle
`Base`](https://speckle.guide/dev/base.html)

It also provides two means to define `Element`s and `Relationship`s:
1. A plugin to [McNeel's Rhinoceros3D Grasshopper]
   (https://www.rhino3d.com/6/new/grasshopper/), that enables both automated and
   manual assignment of `Relationship`s to geometries before their export to
   Speckle; and
2. Manual assignment of `Relationship`s to `Element`s once they have been loaded
   from a Speckle stream in *Rangekeeper*.

In [1]:
from IPython.display import IFrame
IFrame("https://speckle.xyz/embed?stream=3073b96e86&commit=604bea8cc6&c=%5B41.608%2C46.98512%2C69.78354%2C20.86703%2C15.4694%2C16.57642%2C0%2C1%5D",width=700,height=500)